In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_369924/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_36_9,0.994159,0.271838,-0.994724,0.994728,0.024312,4.869224,7.135013,0.029861,0.318604,0.155924,1.373809,0.158380,137.433540,278.768712,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_28_9,0.993971,0.328443,-0.691751,0.980449,0.025094,4.490703,6.051295,0.082384,0.192533,0.158412,1.385831,0.160907,137.370232,278.705404,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_28_8,0.993970,0.331291,-0.679955,0.980647,0.025100,4.471662,6.009103,0.081549,0.194344,0.158430,1.385923,0.160926,137.369758,278.704930,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_29_0,0.993816,0.314519,-0.750257,0.997799,0.025739,4.583816,6.260569,0.020247,0.187252,0.160435,1.395749,0.162961,137.319471,278.654644,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_37_0,0.993710,0.280646,-0.938529,0.979732,0.026181,4.810322,6.934006,0.071292,0.283273,0.161806,1.402544,0.164355,137.285423,278.620595,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,model_1_2_4,0.897582,0.655179,0.625909,0.919512,0.426319,2.305819,1.338101,0.265591,0.932012,0.652931,7.554772,0.663214,131.705135,273.040307,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
471,model_1_2_3,0.886815,0.657192,0.652437,0.901948,0.471134,2.292358,1.243214,0.323550,0.930500,0.686392,8.243816,0.697202,131.505225,272.840398,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
472,model_1_2_2,0.884925,0.655310,0.650571,0.901750,0.479004,2.304946,1.249887,0.324201,0.939410,0.692101,8.364818,0.703001,131.472093,272.807266,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
473,model_1_2_1,0.882979,0.653638,0.649464,0.901271,0.487101,2.316125,1.253847,0.325784,0.948188,0.697926,8.489314,0.708918,131.438567,272.773740,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
